# EOPF S2 MSI L1 A/B Products Data Structure Proposal


In [4]:
from EOProductDataStructure import EOProductBuilder, EOVariableBuilder, EOGroupBuilder
import yaml

# Display function

In [5]:
def set_variables(yaml_group, eopf_group):
    variables = yaml_group['variables']
    for var in variables:
        variable = EOVariableBuilder(var, default_attrs=True)
    
        try:
            variable.dtype = variables[var].split('->')[1]
        except:
            variable.dtype = "string"
            pass
        variable.dims = [d.split('->')[0] for d in variables[var].replace(' ','').replace('F(','').replace(')','').split(',')]
        eopf_group.variables.append(variable)
        for d in variable.dims:
            if d not in eopf_group.dims:
                eopf_group.dims.append(d)

# 1. Read S1 MSI Product

In [6]:
path_product="data/s2_msi_l1ab.yaml"

product = None

with open(path_product, "r") as stream:
    try:
        product = yaml.safe_load(stream)['product']
    except yaml.YAMLError as exc:
        print(exc)
        
s2_msi = EOProductBuilder("S2_MSI_L1AB__", coords=EOGroupBuilder('coords'))

for key, values in product.items():
    
    if key == "attributes":
        s2_msi.attrs = values        
    else:
        group = EOGroupBuilder(key)
        if "groups" in values:
            groups = values['groups']
            for ygp in groups:
                eopf_group = EOGroupBuilder(ygp)
                set_variables(values['groups'][ygp], eopf_group)
                group.groups.append(eopf_group)
        else:
            set_variables(values,group)
            
        s2_msi.groups.append(group)
        
s2_msi.compute()   